In [1]:
from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage
from llama_index.core import Document
from llama_index.core import download_loader, SimpleDirectoryReader, VectorStoreIndex
from llama_index.readers.file import CSVReader, PandasCSVReader
import os
import openai
import pandas as pd

/Users/chetanranpariya/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [15]:
from llama_index.core import Settings

folder_path = './'

with open(f'{folder_path}OpenAI_API_Key.txt', 'r') as f:
    openai.api_key = f.read().strip()

llm = OpenAI(
    model="gpt-4o-mini"
)

Settings.llm = llm

In [3]:
fasion_df = pd.read_csv(f'{folder_path}dataset/myntra_dataset.csv')

In [4]:

documents = [
    Document(
        text=row['name'],
        metadata={
            "id": row['p_id'],
            "description": row['description'],
            "img": row['img'],
            "price": row['price'],
            "color": row['colour'],
            "brand": row['brand']
        },
    )
    for _, row in fasion_df.iterrows()
]

In [5]:
from llama_index.core import StorageContext, load_index_from_storage
database_path = f"{folder_path}/database"
store_index = True
try:
    print("loading index from the local storage")
    storage_context = StorageContext.from_defaults(persist_dir=database_path)
    index = load_index_from_storage(storage_context=storage_context, embed_model=Settings.embed_model)
except:
    print("Creating index from documents")
    index =  VectorStoreIndex.from_documents(documents)
    if store_index:
        index.storage_context.persist(persist_dir=database_path)


loading index from the local storage


In [6]:
from llama_index.core.retrievers import VectorIndexRetriever

retriever = VectorIndexRetriever(index=index, similarity_top_k=5)

#### Using Retriever Engine

In [7]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine(
    retriever = retriever,
)
query_engine

In [8]:
query_engine_response = query_engine.query("Stylish outfit for winter season.")

print(query_engine_response)

For a stylish winter outfit, consider layering a trendy jacket with a cozy sweater. A fashionable option could be the iki chic Women Stylish Black Washed Jacket paired with the KASSUALLY Women Beautiful Off-White Solid Cardigan Set. This combination offers warmth and elegance, perfect for chilly days. Alternatively, you could opt for the FREAKINS Women Lovely Olive Solid Jacket or the FREAKINS Women White Printed Jacket, both of which provide a chic look while keeping you warm. Don't forget to accessorize to complete your winter ensemble!


In [41]:
from IPython.display import display, HTML

def display_response(response):
    for node in response.source_nodes:
        print(node.metadata['description'], end="\n")
        display(HTML(f"<img src='{node.metadata['img']}' style='width:200px;height:200px'></img>"))

### Using default Query Engine

In [21]:
default_engine = index.as_query_engine()

default_engine_response = default_engine.query("Stylish outfit for winter season.")

default_engine_response

Response(response='Consider a cozy off-white cardigan set that features a V-neck and long sleeves, perfect for layering during the winter. Alternatively, a trendy black sleeveless jacket with a stand collar can add a fashionable touch to your winter wardrobe. Both options are stylish and will keep you warm.', source_nodes=[NodeWithScore(node=TextNode(id_='40c1a5c4-a0e5-4e7a-a415-b29465e9aec6', embedding=None, metadata={'id': 16055322, 'description': 'Revamp your winter-wear wardrobe with this elegant sweater. This sweater comes with a cosy V-neck and lovely long sleeves.,Beautiful off-white tone,Solid pattern,V-neck,Long sleeves,Cotton,machine wash,Trend Alert,Youthful and trendy,cardigan sets comprise a cardigan sweater along with a matching top,camisole,or a cropped top to wear underneath.Cotton,Machine washThe model (height 58) is wearing a size S', 'img': 'http://assets.myntassets.com/assets/images/16055322/2021/11/29/da98be2c-7660-43a1-80ac-dbf2a3e23e481638187896390-KASSUALLY-Wome

In [45]:
display_response(default_engine_response)

Revamp your winter-wear wardrobe with this elegant sweater. This sweater comes with a cosy V-neck and lovely long sleeves.,Beautiful off-white tone,Solid pattern,V-neck,Long sleeves,Cotton,machine wash,Trend Alert,Youthful and trendy,cardigan sets comprise a cardigan sweater along with a matching top,camisole,or a cropped top to wear underneath.Cotton,Machine washThe model (height 58) is wearing a size S


Stay warm and look fashionable in this trendy jacket. Tailored with a stand collar and sleeveless design,it makes for a great outfit companion.,Stylish black colour,Washed pattern,Stand collar,Sleeveless,Cotton,dry clean,Material : 100% cotton denim,Wash Care : Dryclean onlyThe model (height 58) is wearing a size S


### Using default chat engine from index.

In [17]:
chat_engine = index.as_chat_engine()

In [18]:
chat_response = chat_engine.query("stylish sleeveless tops")

print(chat_response)

Here are a couple of stylish sleeveless tops you might like:

1. **SASSAFRAS Women Classy Beige Solid Bra Top and Vest**: This top features a halter neck design and is made from a comfortable cotton blend, making it both stylish and comfortable.

2. **Sera Women Stylish Black Solid Puff Sleeves Top**: Although this option has three-quarter puff sleeves, it offers a trendy look with its V-neck design, making it a fashionable choice for various occasions.

If you're looking for more specific styles or colors, let me know!


In [42]:
display_response(chat_response)

Upgrade your casual style by flaunting this fashionable top. Designed with a beautiful solid pattern and halter neck,this top makes you look distinct.,Classy beige tone,Classic solid pattern,Halter neck,Sleeveless,Cotton blend,machine wash,Trend Alert,Bra tops and vests are versatile,stylish,and classy pieces of clothing that ignite interest and play with various details,like cutouts,tie-ups,halter neck,spaghetti strap,plunge neck,and so on. They come in eye-catching prints and patterns,such as sunflowers,joyful motifs,positive slogans,self-care,and more.96% Cotton 4% SpandexThe model (height 58) is wearing a size S


Steal the spotlight when you wear this trendy and versatile top for a get-together with pals. Featuring an attractive solid pattern and V-neck,this top is sure to emphasize your overall appeal.,Black colour,Classic solid pattern,V-neck,Three-quarter puff sleeves,Polyester,machine wash,,Trend Alert,Puff sleeves are typically gathered at the shoulder and puffed in between. They then gather back at the end creating a balloon-like look. Comfortable and stylish,they have been in vogue since the 14th century.Polyester,Machine washThe model (height 58) is wearing a size S


### Using Condense Question chat engine.

In [35]:
from llama_index.core.chat_engine import ContextChatEngine,CondenseQuestionChatEngine

context_chat_engine = CondenseQuestionChatEngine.from_defaults(
    retriever=retriever,
    llm=llm,
    query_engine=query_engine
)

In [36]:
context_chat_response = context_chat_engine.chat("stylish sarees in red and pink color")

print(context_chat_response)

Here are some stylish sarees in red and pink colors:

1. **Satrani Women Red & Pink Geometric Printed Saree**
   - Price: ₹4974
   - Fabric: 100% Chiffon
   - Description: Geometric printed saree with a printed border. Comes with an unstitched blouse piece.
   - ![Image](http://assets.myntassets.com/assets/images/16895216/2022/2/9/e399c8ff-92c0-46c0-85f8-891c6fca9e381644388654751-Satrani-Women-Red--Pink-Geometric-Printed-Saree-480164438865-1.jpg)

2. **Fashionuma Pink & Red Ethnic Motifs Pure Silk Saree**
   - Price: ₹2999
   - Fabric: Pure Silk
   - Description: Ethnic motifs printed saree with a printed border. Comes with an unstitched blouse piece.
   - ![Image](http://assets.myntassets.com/assets/images/19040158/2022/7/11/cf868b56-f3a1-4e52-befb-309dc181fba91657521535792SareeBlouse1.jpg)

3. **Sangria Pink & Red Floral Organza Saree**
   - Price: ₹3399
   - Fabric: Organza
   - Description: Floral printed saree with no border. Comes with an unstitched blouse piece.
   - ![Image](ht

In [43]:
display_response(context_chat_response)

, Design Details Pink and red saree , Ethnic motifs printed saree with printed borderThe saree comes with an unstitched blouse piece,The blouse worn by the model might be for modelling purpose only. Check the image of the blouse piece to understand how the actual blouse piece looks like.Length: 5.5 metres plus 0.8 metre blouse piece ,Width: 1.06 metres (approx.)Pure Silk,Dry Clean


, Design Details Red and pink saree , Geometric printed saree with printed borderThe saree comes with an unstitched blouse piece,The blouse worn by the model might be for modelling purpose only. Check the image of the blouse piece to understand how the actual blouse piece looks like.Length: 5.5 metres plus 0.8 metre blouse piece ,Width: 1.06 metres (approx.)Saree:100% Chiffon,Blouse : 100% Chiffon,Dry Clean


, Design Details Grey and pink saree , Floral embellished saree with embellished border , Has beads and stones detailThe saree comes with an unstitched blouse piece,The blouse worn by the model might be for modelling purpose only. Check the image of the blouse piece to understand how the actual blouse piece looks like.,Length: 5.5 metres plus 0.8 metre blouse piece ,Width: 1.06 metres (approx.)


,Design Details:,Pink and Orange checked saree and has a woven design border with tasselled detail,Blouse Piece,The model is wearing a stitched version of the blouse. The saree comes with an unstitched blouse piece.Length: 5.5 metres plus 80 cm blouse piece,Width: 1 metre (approx)Saree fabric: silk cotton,Blouse fabric: silk cotton,Hand-wash


Breathe life into your ethnic collection by adding this elegant-looking saree. Designed with a floral print,woven border,and a silk blend fabric,this piece is sure to add to your appeal. ,Design Details ,Pink and blue Dyedbanarasi saree,Floral printed saree with woven design border,Trend Alert,Soft,light,and soothing,pastels always make a comeback in the spring and summer seasons. Encompassing a palette of hues including delicate pinks,pale lilac,cool blues,and mint tints,pastels exude a fresh,youthful vibe.Saree fabric:  Silk Blend ,Blouse fabric:  Silk Blend ,Dry cleanLength: 5.5 metres plus 0.8 metre(80 cm) blouse piece,Width: 1.06 metres (approx.)


In [38]:
chat_history= [ChatMessage(role="user", content="stylish sarees in red and pink color")]

#### Trying to continue conversation using chat history.

In [39]:

context_chat_response = context_chat_engine.chat("Can you suggest some more options please?", chat_history)

print(context_chat_response)

Here are some stylish saree options in red and pink colors:

1. **Satrani Women Red & Pink Geometric Printed Saree**
   - Description: Geometric printed saree with a printed border.
   - Fabric: 100% Chiffon.
   - Length: 5.5 metres plus 0.8 metre blouse piece.
   - Price: ₹4974.
   - ![Image](http://assets.myntassets.com/assets/images/16895216/2022/2/9/e399c8ff-92c0-46c0-85f8-891c6fca9e381644388654751-Satrani-Women-Red--Pink-Geometric-Printed-Saree-480164438865-1.jpg)

2. **Fashionuma Pink & Red Ethnic Motifs Pure Silk Saree**
   - Description: Ethnic motifs printed saree with a printed border.
   - Fabric: Pure Silk.
   - Length: 5.5 metres plus 0.8 metre blouse piece.
   - Price: ₹2999.
   - ![Image](http://assets.myntassets.com/assets/images/19040158/2022/7/11/cf868b56-f3a1-4e52-befb-309dc181fba91657521535792SareeBlouse1.jpg)

These options combine vibrant colors with unique designs, making them stylish choices for various occasions.


In [44]:
display_response(context_chat_response)

, Design Details Pink and red saree , Ethnic motifs printed saree with printed borderThe saree comes with an unstitched blouse piece,The blouse worn by the model might be for modelling purpose only. Check the image of the blouse piece to understand how the actual blouse piece looks like.Length: 5.5 metres plus 0.8 metre blouse piece ,Width: 1.06 metres (approx.)Pure Silk,Dry Clean


, Design Details Red and pink saree , Geometric printed saree with printed borderThe saree comes with an unstitched blouse piece,The blouse worn by the model might be for modelling purpose only. Check the image of the blouse piece to understand how the actual blouse piece looks like.Length: 5.5 metres plus 0.8 metre blouse piece ,Width: 1.06 metres (approx.)Saree:100% Chiffon,Blouse : 100% Chiffon,Dry Clean


, Design Details Grey and pink saree , Floral embellished saree with embellished border , Has beads and stones detailThe saree comes with an unstitched blouse piece,The blouse worn by the model might be for modelling purpose only. Check the image of the blouse piece to understand how the actual blouse piece looks like.,Length: 5.5 metres plus 0.8 metre blouse piece ,Width: 1.06 metres (approx.)


,Design Details:,Pink and Orange checked saree and has a woven design border with tasselled detail,Blouse Piece,The model is wearing a stitched version of the blouse. The saree comes with an unstitched blouse piece.Length: 5.5 metres plus 80 cm blouse piece,Width: 1 metre (approx)Saree fabric: silk cotton,Blouse fabric: silk cotton,Hand-wash


Breathe life into your ethnic collection by adding this elegant-looking saree. Designed with a floral print,woven border,and a silk blend fabric,this piece is sure to add to your appeal. ,Design Details ,Pink and blue Dyedbanarasi saree,Floral printed saree with woven design border,Trend Alert,Soft,light,and soothing,pastels always make a comeback in the spring and summer seasons. Encompassing a palette of hues including delicate pinks,pale lilac,cool blues,and mint tints,pastels exude a fresh,youthful vibe.Saree fabric:  Silk Blend ,Blouse fabric:  Silk Blend ,Dry cleanLength: 5.5 metres plus 0.8 metre(80 cm) blouse piece,Width: 1.06 metres (approx.)
